In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow
from tensorflow import keras

2022-11-10 23:30:53.495825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Multiple outputs in cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
#data needs to be downloaded into an updating-csv file on disk
path = os.getcwd()
file = '/cryptodata.csv'

data = pd.read_csv(path+file)

In [15]:
data.head()
data.shape

,time,low,high,open,close,volume
0,2021-06-01 01:00:00,2656.33,2733.17,2733.17,2659.90,19736.222380
1,2021-06-01 02:00:00,2615.90,2675.85,2659.89,2645.90,18937.228584
2,2021-06-01 03:00:00,2615.06,2679.00,2645.56,2631.28,16276.604299
3,2021-06-01 04:00:00,2616.62,2659.88,2630.66,2637.46,7452.697987
4,2021-06-01 05:00:00,2633.71,2714.16,2636.74,2699.64,11836.470941


(12237, 6)

In [4]:
good_buys = pd.DataFrame(data[data['Gain?'] == 1])
bad_buys  = pd.DataFrame(data[data['Gain?'] == 0])

seed = random.randint(100)
y = np.concatenate((np.ones(len(good_buys)), np.zeros(len(bad_buys))))
x = data.drop(['Gain?', 'Coin'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = seed, stratify=y)

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

numBranches = X_train.shape[1]

NameError: name 'data' is not defined

In [26]:
def build_model(neurons, drop):
    
    model = Sequential()
    
    opt = keras.optimizers.Adam(lr=0.001)
    
    act = "relu"

    model.add(Dense(neurons[0], input_shape=(numBranches,), activation=act))

    for i in range(1, len(neurons) - 1):
        model.add(Dense(neurons[i], activation=act))
        model.add(Dropout(drop, seed=seed))

   
    model.add(
        Dense(network[len(neurons)-1], activation="sigmoid"))

    model.compile(
        loss="binary_crossentropy",
        optimizer=opt,
        metrics=tf.keras.metrics.Precision(),
    )
    return model


In [27]:
def running(neurons, drop, layer, batch, xtrain, xtest, ytrain, ytest):
    network = []
    numEpochs = 150  # Number of times the NN gets trained.
    batchSize = BATCH
    numLayers = LAYER
    neurons = numBranches
    
    i = 0

    # This creates a list that has the structure of the NN.
    for i in range(numLayers):
        network.append(neurons)
    network.append(1)
    numNeurons = sum(network)

    # This is a conformation that the script is starting and the NN structure is displayed.
    print("Script starting....\n", network)

    
    sufix = ".CPU"
    print("CPU")

    # Start time for file name.
    startTime = datetime.now()
    pre = time.strftime("%Y.%m.%d_") + time.strftime("%H.%M.%S.")

    # Filename for keras model to be saved as.
    h5name = (
        "numLayers"
        + str(LAYER)
        + ".numBr anches"
        + str(neurons)
        + ".batchSize"
        + str(BATCH)
    )
    
    #determines user path to generate a model
    
    path = os.getcwd()
    
    modelName = path + '/' + pre + h5name + sufix + ".h5"

    # Filename for plots to be identified by saved model.
    figname = path + '/' + pre + ".plots"

    # Using model and setting parameters.
    model = build_model(network,RATE)
    
    model.save(modelName)

    # This checkpoint is used for recovery of trained weights incase of interuption.
    checkPointsCallBack = ModelCheckpoint("temp.h5", save_best_only=True)

    # This terminates early if the monitor does not see an improvement after a certain
    # amount of epochs given by the patience.
    earlyStopCallBack = EarlyStopping(
        monitor="val_loss", patience=30, restore_best_weights=True
    )
        

    # This is where the training starts.
    kModel = model.fit(
        xtrain,
        ytrain,
        epochs=numEpochs,
        batch_size=batchSize,
        validation_data=(xtest, ytest),
        verbose=1,
        callbacks=[earlyStopCallBack, checkPointsCallBack]
    )
    
    return model,kModel,startTime,modelName
    
    

In [28]:
def checkTraining(model,kModel):
    # This is the predicted score. Values range between [0,1]
    y_predicted = model.predict(X_test)

    # Prediction, fpr,tpr and threshold values for ROC.
    fpr, tpr, thresholds = roc_curve(y_test, y_predicted)
    aucroc = auc(fpr, tpr)
    precision, recall, thresRecall = precision_recall_curve(y_test, y_predicted)
    
    figure, axs = plt.subplots(2, figsize=(6,10))
    axs[0].set_xlabel("Score")
    axs[0].set_ylabel("Distribution")
    axs[0].legend(loc="upper right")
    axs[0].plot(fpr, tpr, "r-", label="ROC (area = %0.6f)" % (aucroc))
    axs[0].plot([0, 1], [0, 1], "--", color=(0.6, 0.6, 0.6), label="Luck")
    axs[0].set_xlim([-0.05, 1.05])
    axs[0].set_ylim([-0.05, 1.05])
    axs[0].set_xlabel("False Positive Rate")
    axs[0].set_ylabel("True Positive Rate")
    axs[0].set_title("Receiver operating characteristic")
    axs[0].legend(loc="lower right")
    axs[0].grid(True)
    


    # AUC
    #plotROC(fpr, tpr, aucroc)

    axs[1].plot(thresRecall, precision[:-1], "b--", label="Precision")
    axs[1].plot(thresRecall, recall[:-1], "g-", label="Recall")
    axs[1].set_ylim([0.00, 1.05])
    axs[1].set_xlabel("Threshold")
    axs[1].set_title("Precision/Recall vs. Threshold Curve")
    axs[1].legend(loc="lower right")
    axs[1].grid(True)

    compare_train_test(kModel, X_train, y_train, X_test, y_test)
    
    return aucroc

In [29]:
def compare_train_test(kModel, X_train, y_train, X_test, y_test, bins=100):
    """
    This creates the signal and background distrubution.
    """
    i = 0
    j = 0
    sig_index = []
    bkg_index = []
    decisions = []
        
    for X, y in ((X_train, y_train), (X_test, y_test)):
        # captures indices in X_train and X_test dataframes that correlate to signal and background events
#         while i < len(y):
#             if y[i] == 1.:
#                 sig_index.append(j)
#             elif y[i] == 0.:
#                 bkg_index.append(j)
#             i += 1
#             j += 1
#         i = 0
        d1 = model.predict(X[y > 0.5]).ravel()  # signal
        d2 = model.predict(X[y < 0.5]).ravel()  # background
        decisions += [d1, d2]
                
    low = min(np.min(d) for d in decisions)
    high = max(np.max(d) for d in decisions)
    low_high = array([low, high])
    
    
    
    train_s = decisions[0]
    train_b = decisions[1]
    test_s = decisions[2]
    test_b = decisions[3]
        
    
    #Combining scores for test and training sets of signal and background seperately
    S_scores = np.concatenate((train_s, test_s), axis=None)
    B_scores = np.concatenate((train_b, test_b), axis=None)
    max_s = S_scores.max()    
    

    #Scoring histograms for training sets
    
    figure2, axes = plt.subplots(2, figsize=(16,20))
    axes[0].hist(
        train_s,
        color="r",
        alpha=0.5,
        range=low_high,
        bins=bins,
        histtype="stepfilled",
        density=True,
        label="S (train)",
    )
    
    axes[0].hist(
        train_b,
        color="b",
        alpha=0.5,
        range=low_high,
        bins=bins,
        histtype="stepfilled",
        density=True,
        label="B (train)",
    )
    
    #Scoring points for testing sets

    histS, bins = np.histogram(test_s, bins=bins, range=low_high, density=True)
    scale = len(test_s) / sum(histS)
    err = np.sqrt(histS * scale) / scale

    width = bins[1] - bins[0]
    center = (bins[:-1] + bins[1:]) / 2
    axes[0].errorbar(center, histS, yerr=err, fmt="o", c="r", label="S (test)")

    histB, bins = np.histogram(test_b, bins=bins, range=low_high, density=True)
    scale = len(test_b) / sum(histB)
    
    histS2, bins = (np.histogram(S_scores, bins=bins, range=low_high, density=True))
    histB2, bins = (np.histogram(B_scores, bins=bins, range=low_high, density=True))
    
    #start = score_testing(histS2, histB2)
    
    err = np.sqrt(histB * scale) / scale
    axes[0].set_title("Net Learning Feedback")
    axes[0].errorbar(center, histB, yerr=err, fmt="o", c="b", label="B (test)")
    #axes[0].vlines(start, 0, 30, colors='green', label='Score Threshold')
    axes[0].legend(loc='upper right')
    axes[0].set_xlim(0,1)

In [31]:
batch = 512

layers = 5

# This runs the training. A for loop can be used to vary the parameters. 
model,kModel,startTime,modelName=running(layers,batch,0.5, X_train, y_train, X_test, y_test) 

NameError: name 'X_train' is not defined